<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Calculation_concentration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Concentration calculation
This notebook is used to calculate the concentration of phtalates in an environemental sample using an internal standard for quantitation. The Calibration_calculation.ipynb notebook, yields severall of the required files nescessary for the calculations in this notebook. The files *Cal_{n}_analyte_linreg.csv* need to be used which is located in the *calibration_output* folder on the google drive. If this notebook is not applied from my machine in connection with my google drive, either the path to the required files needs to be adabted or changed.
The following code allows for the calculation of the concentration, by using the linear regression form a calibration series and the ratio of the signal of the analyte with the internal standard (area). All the data regarding the calibration are present on my google drive in the folders: *calibration_input* and *calibration_output*, while a new google drive folder will be used to save the final data in as well as a new input folder.

*Google drive folder:*
quantitation_input
quantitation_output

The .csv files containing the peak list from the sample should be saved in the in the input folder. The names should follow a consequent naming sceme:

***Sample_{date_of_analysis}_{tag_bonus_info}_{Sample_Nr}.csv***

Here the Sample number *(Sample_Nr)* is of key importance and needs to be properly documented in the Labjournal, as it is the main way of differentiation.


The following code is used to import all the nescessary libraries and mount the google drive.

In [ ]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import os

drive.mount('/content/drive', force_remount = True)

sample_nr_low = 1 #needs to be addapted to 1 nr in the series
sample_nr_high =2 #needs to be addapted to the last nr of the series

The following code is the addapted code from the Calibration_calculation.ipynb and yields the separated in separated .csv files, which will be used to calculate the ratio in a following code segment.

The output file has the following filename and is saved in the quantitation_output folder:

***Sample_{n}_row{i+1}.csv***

In [ ]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
    files = glob.glob(f'/content/drive/My Drive/quantitation_input/Sample_*{n}.csv')

    if not files:
        print(f'Files matching pattern Sample Nr. {n} was not found.')
        break

    else:
        # Loop over each row and save a separate data frame for each
        df = pd.read_csv(files[0], header=4)
        num_rows = len(df)
        
        for i in range(num_rows):
            row_dfs = []
            for file in files:
                df = pd.read_csv(file, header=4, na_values='')

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i, :].dropna().astype(str)
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/quantitation_output/Sample_{n}_row{i+1}.csv', index=False, header=False)
            
            # Convert all values to strings before joining and print the row in the desired format
            formatted_row = '\t'.join(map(str, combined_df.values[0]))
            print(formatted_row)

The following code allows to calculate the ratio of the analytes with the internal standard. The output is saved as:

***Sample_{n}_row*_ratio.csv**

In [ ]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
  divisor_file = f'/content/drive/My Drive/quantitation_output/Sample_{n}_row1.csv'
  directory = '/content/drive/My Drive/calibration_output'

  divisor_row = pd.read_csv(divisor_file, skiprows=1, header=None)

  csv_files = glob.glob(os.path.join(directory, '*_row*.csv'))

  # Process each CSV file
  for csv_file in csv_files:
    if csv_file == divisor_file:
        continue  # Skip the divisor file itself

    data_df = pd.read_csv(csv_file, skiprows=1, header=None)
  
    # Perform division on the data
    divided_data_df = data_df.div(divisor_row, axis='columns')

    # Create a new file name with the "_ratio" tag
    new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv'

    divided_data_df.to_csv(new_file_name, sep='\t', header=False, index=False)
     

The following code takes the ratio files as well as the parameters for the linear regression (analyte specific), and yields a final output file that contains all the information, such as the information of the ratio used for the quantitaiton, the parameters for the linear equation as well as other information such as compound id and other general sample information. 

The final outpur files have the following name format:

***Sample_{n}_quantitation_output.csv***
